# Module _gameoflife_ 

This module contains the core functions needed to generate the initial distribution of the cells, i.e. the initial pattern, and their future evolution controlled by the _Game of life_ rules. \
The module is divided into two submodules, which are _patterns_ and _evolution_.

## Submodule _evolution_

The module _evolution_ consists in two functions, namely _newgen_ and _evolution_. They both deal with 2D numpy arrays, which contains boolean variables: `True` stands for alive cells and `False` stands for dead cells. These numpy arrays represent the 2D grid of the _Game of life_ world in which cells live. 

### Function _newgen_

The function _newgen_ takes in input the current generation of cells and gives in output the next one, calculated through the rules of _Game of life_. Its structure is very simple. 


First, the matrix representing the current generation of cells gets padded in order to identify the borders of the corresponding grid, making a toroidal structure of the world in which they live. Then, since alive and dead cells have to respect different rules, the function creates two different arrays for their indeces: `alive_idx` and `dead_idx`. However, these arrays are traslated with respect to the original grid due to the padding procedure; hence, the function fix this by adding a traslation by [1, 1]. 


Ultimately, there is the actual next generation creation. The old generation matrix gets copied into a new one, which is then modified by the two for cicles that apply the _Game of life_ rules to each cell. The first for cicle make alive cells evolve, whereas the second make the dead cells evolve. 

In [1]:
import numpy as np
import numpy.typing as npt #For writing the data types in the function definition

def newgen(cells: npt.NDArray[np.bool_]):
    #This create a wrapped surface, where top is identified with bottom and left with right
    padded = np.pad(cells, pad_width=1, mode='wrap')
    
    #Extracts the indexes of living cells (True) and dead cells (False)
    alive_idx = np.argwhere(cells == True) 
    dead_idx = np.argwhere(cells == False) 

    #I add [1, 1] in order to traslate the indexes and make them compatible with the padded matrix
    alive_idx += np.array([1, 1])
    dead_idx += np.array([1, 1])

    #Make a copy of the grid, i.e. the next generation (necessary to make the code work)
    newgen = np.copy(cells)

    #Calculates the evolution of the living cells
    for index in alive_idx:
        neig = padded[index[0]-1:index[0]+2, index[1]-1:index[1]+2]
        #The -1 is to delete the cell I am considering from the counts of alive neighbors
        nalive = len(neig[neig == True]) - 1
        if (nalive != 2 and nalive != 3):
            newgen[index[0]-1, index[1]-1] = False

    #Calculates the evolution of the dead cells
    for index in dead_idx:
        neig = padded[index[0]-1:index[0]+2, index[1]-1:index[1]+2]
        #The -1 is to delete the cell I am considering from the counts of alive neighbors
        nalive = len(neig[neig == True]) 
        if (nalive == 3):
            newgen[index[0]-1, index[1]-1] = True

    return newgen

### Function _evolution_

This function takes in input the first generation, i.e. the original pattern, and gives in output the evolution timeline of that original pattern. The number of steps in the timeline is set by the other parameter of the _evolution_ function, which is `timesteps`.  

In [2]:
def evolution(genzero: npt.NDArray[np.bool_], timesteps: int):
    
    # Creates a list containing the configurations for each timestep in the evolution
    timeline = []
    
    # Initialize the current state with the generation zero
    current_state = genzero
    timeline.append(current_state) # Include the starting state in the timeline
    
    # Calculates the configuration for each timestep
    for t in range(timesteps):
        # ERROR FIX: We must use 'current_state' as input, not 'genzero' repeatedly
        new = newgen(cells=current_state)
        
        # Update the current state for the next iteration
        current_state = new
        
        timeline.append(new)
    
    return timeline

## Submodule _patterns_